In [1]:
import os
import numpy as np
import pandas as pd
import rasterio
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor

SEQ_LEN = 6                 
HORIZONS = 3               
PATCH_SIZE = 13             
HALF = PATCH_SIZE // 2
FILL_NAN_VALUE = 0.0

REQUIRED_COLS = [
    "era5_t2m_file", "era5_d2m_file", "era5_tp_file",
    "era5_u10_file", "era5_v10_file",
    "viirs_file", "dem_file", "lulc_file"
]

In [2]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [3]:
def _load_single_raster(path):
    with rasterio.open(path) as src:
        arr = src.read() 

    if arr.shape[0] == 1:
        
        return arr[0]
    else:
     
        return arr


In [4]:
def load_rasters(df, raster_cols, max_workers=8):
    
    all_paths = set()

    for col in raster_cols:
        if col in df.columns:
            all_paths.update(df[col].dropna().unique())
    all_paths = list(all_paths)

    cache = {}
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        results = list(ex.map(_load_single_raster, all_paths))

    for path, arr in zip(all_paths, results):
        if arr is not None:
            cache[path] = arr
    return cache


In [5]:
def _safe_center(h, w, patch_size=PATCH_SIZE):
    half = patch_size // 2
    r = np.clip(h // 2, half, h - half - 1)
    c = np.clip(w // 2, half, w - half - 1)
    return r, c

In [6]:
def _extract_patch(arr, row, col, patch_size=PATCH_SIZE):
    half = patch_size // 2
    h, w = arr.shape

    r0 = row - half
    r1 = row + half + 1
    c0 = col - half
    c1 = col + half + 1

    patch = np.zeros((patch_size, patch_size), dtype=arr.dtype)

    r0_clip = max(r0, 0)
    r1_clip = min(r1, h)
    c0_clip = max(c0, 0)
    c1_clip = min(c1, w)

    pr0 = r0_clip - r0
    pr1 = pr0 + (r1_clip - r0_clip)
    pc0 = c0_clip - c0
    pc1 = pc0 + (c1_clip - c0_clip)

    patch[pr0:pr1, pc0:pc1] = arr[r0_clip:r1_clip, c0_clip:c1_clip]

    return patch

In [7]:
def build_sample(seq_rows, horizon_rows, cache, force_fire=False):
    seq_patches = []

   
    for _, row in seq_rows.iterrows():
        bands = []
        for var in ["era5_t2m_file", "era5_d2m_file", "era5_tp_file",
                    "era5_u10_file", "era5_v10_file"]:
            arr = cache[row[var]]

            if len(arr.shape) == 3:
                arr = arr[0]

            h, w = arr.shape
            r, c = _safe_center(h, w)
            bands.append(_extract_patch(arr, r, c))

        dem = cache[row["dem_file"]]
        lulc = cache[row["lulc_file"]]

        if len(dem.shape) == 3:
            dem = dem[0]
        if len(lulc.shape) == 3:
            lulc = lulc[0]

        h, w = dem.shape
        r, c = _safe_center(h, w)
        bands.append(_extract_patch(dem, r, c))
        bands.append(_extract_patch(lulc, r, c))

        seq_patches.append(np.stack(bands, axis=-1))

    X = np.stack(seq_patches, axis=0)


    horizon_patches = []
   
    for _, row in horizon_rows.iterrows():
        viirs_stack = cache[row["viirs_file"]]
        
    
        target_band_idx_list = eval(row["target_band_idxs"])
 
        idx = target_band_idx_list[0]
        
        band = viirs_stack[idx - 1]
        h, w = band.shape
        r, c = _safe_center(h, w)

        if force_fire and np.any(band > 0):
            fire_pos = np.argwhere(band > 0)
            r, c = fire_pos[np.random.randint(len(fire_pos))]

        horizon_patches.append(_extract_patch(band, r, c))

    y = np.stack(horizon_patches, axis=0)

    return X.astype("float32"), y.astype("float32")

In [24]:
import numpy as np
import pandas as pd
import tensorflow as tf

def make_generator(df, cache, fire_ratio=0.5):
    
    valid_start_indices = list(range(len(df) - SEQ_LEN - HORIZONS + 1))

    fire_start_indices = []
    non_fire_start_indices = []
    
    for i in valid_start_indices:
        horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        # This line iterates through DataFrame rows to check for fire, which is fine.
        has_fire = any(np.any(cache[row["viirs_file"]] > 0) for _, row in horizon_rows.iterrows())
        
        if has_fire:
            fire_start_indices.append(i)
        else:
            non_fire_start_indices.append(i)

    # --- Start of sampling logic (your core strategy) ---
    num_fire_samples = len(fire_start_indices)
    
    if num_fire_samples == 0:
        print("Warning: No fire events found in the dataset. Training will be difficult.")
        # If no fires, take a small sample of non-fire indices
        num_non_fire_samples_to_use = min(len(non_fire_start_indices), 1000)
    else:
        # Calculate how many non-fire samples to use to achieve the desired ratio
        num_non_fire_samples_to_use = int((num_fire_samples / fire_ratio) - num_fire_samples)
        num_non_fire_samples_to_use = min(num_non_fire_samples_to_use, len(non_fire_start_indices))

    # Sample the indices to create a balanced list
    # The crucial change is ensuring all indices are integers.
    fire_indices_to_use = np.array(fire_start_indices, dtype=int)
    non_fire_indices_to_use = np.random.choice(
        non_fire_start_indices,
        size=num_non_fire_samples_to_use,
        replace=False
    ).astype(int)  # Ensure non-fire indices are also integers
    
    indices_to_use = np.concatenate([fire_indices_to_use, non_fire_indices_to_use])
    np.random.shuffle(indices_to_use)
    
    # --- End of sampling logic ---
    
    for i in indices_to_use:
        # 'i' is now guaranteed to be an integer, preventing the TypeError
        seq_rows = df.iloc[i : i + SEQ_LEN]
        horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        
        X, y = build_sample(seq_rows, horizon_rows, cache)
        
        yield X, y

In [8]:
def make_generator(df, cache, fire_ratio=0.5):
    
    valid_start_indices = list(range(len(df) - SEQ_LEN - HORIZONS + 1))

    fire_start_indices = []
    non_fire_start_indices = []
    
    for i in valid_start_indices:
        horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        has_fire = any(np.any(cache[row["viirs_file"]] > 0) for _, row in horizon_rows.iterrows())
        
        if has_fire:
            fire_start_indices.append(i)
        else:
            non_fire_start_indices.append(i)

    
    num_fire_samples = len(fire_start_indices)
    
    if num_fire_samples == 0:
        print("Warning: No fire events found in the dataset. Training will be difficult.")
        
        num_non_fire_samples_to_use = min(len(non_fire_start_indices), 1000) 
    else:
   
        num_non_fire_samples_to_use = int((num_fire_samples / fire_ratio) - num_fire_samples)
        num_non_fire_samples_to_use = min(num_non_fire_samples_to_use, len(non_fire_start_indices))

    # 4. Sample the indices to create a balanced list
    fire_indices_to_use = fire_start_indices
    non_fire_indices_to_use = np.random.choice(
        non_fire_start_indices,
        size=num_non_fire_samples_to_use,
        replace=False 
    )
    
    indices_to_use = np.concatenate([fire_indices_to_use, non_fire_indices_to_use])
    np.random.shuffle(indices_to_use)
    
    for i in indices_to_use:
        seq_rows = df.iloc[i : i + SEQ_LEN]
        horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        
        X, y = build_sample(seq_rows, horizon_rows, cache)
        
        yield X, y

In [22]:
def create_dataset(df, cache, shuffle_buf=256):
    output_signature = (
        tf.TensorSpec(shape=(SEQ_LEN, PATCH_SIZE, PATCH_SIZE, 7), dtype=tf.float32),
        tf.TensorSpec(shape=(HORIZONS, PATCH_SIZE, PATCH_SIZE), dtype=tf.float32),
    )
    
    ds = tf.data.Dataset.from_generator(
        lambda: make_generator(df, cache),
        output_signature=output_signature
    )
    
    ds = ds.shuffle(shuffle_buf, reshuffle_each_iteration=True)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    return ds

In [10]:
if __name__ == "__main__":
    csv_path = r"C:\Users\Ankit\Datasets_Forest_fire\sequence_index_hourly_binary.csv"
    df = pd.read_csv(csv_path)
    
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    TOTAL = len(df)
    VAL_SPLIT = 0.2
    val_size = int(TOTAL * VAL_SPLIT)

    val_df = df.iloc[:val_size].copy()
    train_df = df.iloc[val_size:].copy()

    print(f"Total samples: {TOTAL}")
    print(f"Train samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")

    raster_cols = REQUIRED_COLS
    print("Loading rasters into memory...")
    cache = load_rasters(df, raster_cols, max_workers=8)
    print(f"Loaded {len(cache)} rasters into memory ✅")

    # Use the new, balanced dataset functions
    train_dataset = create_dataset(train_df, cache)
    val_dataset = create_dataset(val_df, cache)

Total samples: 17535
Train samples: 14028
Validation samples: 3507
Loading rasters into memory...
Loaded 9 rasters into memory ✅


In [11]:
BATCH_SIZE = 16
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [25]:
train_dataset_unbatched = create_dataset(
    train_df,
    cache,
    shuffle_buf=1 
)

fire_pixel_counts = {f't+{i+1}': 0 for i in range(HORIZONS)}
total_samples_with_fire = 0

print("Counting fire pixels in the training dataset...")
for X, y in train_dataset_unbatched:
    if np.any(y.numpy() > 0):
        total_samples_with_fire += 1
        for i in range(HORIZONS):
            fire_pixels = np.sum(y.numpy()[i] > 0)
            fire_pixel_counts[f't+{i+1}'] += fire_pixels

print("Finished counting.")

results_df = pd.DataFrame(fire_pixel_counts.items(), columns=['Horizon', 'Number of Fire Pixels'])

print(f"\nTotal number of training samples: {len(train_df)}")
print(f"Total samples containing at least one fire event: {total_samples_with_fire}")
print("\nNumber of fire pixels per horizon in the training dataset:")
print(results_df)

Counting fire pixels in the training dataset...
Finished counting.

Total number of training samples: 14028
Total samples containing at least one fire event: 2654

Number of fire pixels per horizon in the training dataset:
  Horizon  Number of Fire Pixels
0     t+1                  11408
1     t+2                  11408
2     t+3                  11419


In [12]:
from tensorflow.keras import layers, models

In [13]:
SEQ_LEN = 6           
PATCH_H = 13            
PATCH_W = 13          
CHANNELS = 7       
HORIZONS = 3            
LSTM_UNITS = 64    
CNN_FEATURES = 128

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

def build_conv_lstm_unet_model(
    seq_len=SEQ_LEN,
    patch_h=PATCH_H,
    patch_w=PATCH_W,
    channels=CHANNELS,
    horizons=HORIZONS
):
    inp = layers.Input(shape=(seq_len, patch_h, patch_w, channels))

    enc1 = layers.ConvLSTM2D(
        filters=32, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu'
    )(inp)
    enc1_pool = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same')(enc1)

    enc2 = layers.ConvLSTM2D(
        filters=64, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu'
    )(enc1_pool)
    enc2_pool = layers.MaxPooling3D(pool_size=(1, 2, 2), padding='same')(enc2)

    bottleneck = layers.ConvLSTM2D(
        filters=128, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu'
    )(enc2_pool)

    dec1_up = layers.UpSampling3D(size=(1, 2, 2))(bottleneck)
    dec1_up = layers.Conv3D(filters=64, kernel_size=(3,3,3), padding='same', activation='relu')(dec1_up)
    dec1_up_cropped = layers.Cropping3D(cropping=((0, 0), (0, 1), (0, 1)))(dec1_up)
    dec1_concat = layers.Concatenate(axis=-1)([dec1_up_cropped, enc2])

    dec2_up = layers.UpSampling3D(size=(1, 2, 2))(dec1_concat)
    dec2_up = layers.Conv3D(filters=32, kernel_size=(3,3,3), padding='same', activation='relu')(dec2_up)
    dec2_up_cropped = layers.Cropping3D(cropping=((0, 0), (0, 1), (0, 1)))(dec2_up)
    dec2_concat = layers.Concatenate(axis=-1)([dec2_up_cropped, enc1])

    output_convlstm = layers.ConvLSTM2D(
        filters=1, kernel_size=(3, 3), padding='same', return_sequences=True, activation='sigmoid'
    )(dec2_concat[:, :horizons])

    final_output = tf.keras.ops.squeeze(output_convlstm, axis=-1)

    model = models.Model(inputs=inp, outputs=final_output)
    return model

In [15]:
print(train_dataset.element_spec)

(TensorSpec(shape=(None, 6, 13, 13, 7), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3, 13, 13), dtype=tf.float32, name=None))


In [16]:
from tensorflow.keras import callbacks

In [17]:
model = build_conv_lstm_unet_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 6, 13, 13, 7)      │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_lstm2d (ConvLSTM2D)      │ (None, 6, 13, 13, 32)     │          45,056 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d (MaxPooling3D)  │ (None, 6, 7, 7, 32)       │               0 │ conv_lstm2d[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)    │ (None, 6, 7, 7, 64)       │         221,440 │ max_pooling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_1               │ (None, 6, 4, 4, 64)       │               0 │ conv_lstm2d_1[0][0]        │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)    │ (None, 6, 4, 4, 128)      │         885,248 │ max_pooling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling3d (UpSampling3D)  │ (None, 6, 8, 8, 128)      │               0 │ conv_lstm2d_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d (Conv3D)               │ (None, 6, 8, 8, 64)       │         221,248 │ up_sampling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ cropping3d (Cropping3D)       │ (None, 6, 7, 7, 64)       │               0 │ conv3d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 6, 7, 7, 128)      │               0 │ cropping3d[0][0],          │
│                               │                           │                 │ conv_lstm2d_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling3d_1               │ (None, 6, 14, 14, 128)    │               0 │ concatenate[0][0]          │
│ (UpSampling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_1 (Conv3D)             │ (None, 6, 14, 14, 32)     │         110,624 │ up_sampling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ cropping3d_1 (Cropping3D)     │ (None, 6, 13, 13, 32)     │               0 │ conv3d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 6, 13, 13, 64)     │               0 │ cropping3d_1[0][0],        │
│                               │                           │                 │ conv_lstm2d[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 3, 13, 13, 64)     │               0 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 1,485,960 (5.67 MB)

 Trainable params: 1,485,960 (5.67 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()],
)

In [19]:
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)
checkpoint = callbacks.ModelCheckpoint(
    "best_unet_model.h5", # Changed filename
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [20]:
BATCH_SIZE = 16
steps_per_epoch = len(train_df) // BATCH_SIZE
validation_steps = len(val_df) // BATCH_SIZE

print(f"Batch size: {BATCH_SIZE}")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")


Batch size: 16
Steps per epoch: 876
Validation steps: 219


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[early_stop, checkpoint],
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

Epoch 1/50


In [12]:
# Load the entire model from the .h5 file
import tensorflow as tf 
model = tf.keras.models.load_model(r"C:\Users\Ankit\Downloads\best_cnn_lstm_model (3).h5")

In [13]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 6, 13, 13, 7)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 6, 128)              │          94,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cls_out (Dense)                      │ (None, 507)                 │          32,955 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 3, 13, 13)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 176,765 (690.49 KB)

 Trainable params: 176,763 (690.48 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [11]:
import tensorflow as tf

# Assuming your 'val_df' and 'cache' from your data preparation script are in memory.

# --- 1. Load the Model ---
# Ensure the path to your saved model is correct.
model_path = r"C:\Users\Ankit\Downloads\best_cnn_lstm_model (3).h5"
try:
    loaded_model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully!")
    loaded_model.summary()
except Exception as e:
    print(f"Error loading model from {model_path}: {e}")
    # Exit the script if the model cannot be loaded
    exit()

# --- 2. Create the Validation Dataset for Evaluation ---
# Use your existing 'create_dataset' function.
# Do NOT shuffle the dataset for evaluation.
# Do NOT force fire samples as we want to test on the natural distribution.
val_dataset_for_evaluation = create_dataset(
    val_df,
    cache,
    shuffle=False,
    ensure_fire=False
)

# Batch the dataset using the same batch size as training.
BATCH_SIZE = 2 # Change this to your training batch size if different
val_dataset_for_evaluation = val_dataset_for_evaluation.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# --- 3. Evaluate the Model on the Validation Dataset ---
print("\nEvaluating model performance on the validation set...")
evaluation_results = loaded_model.evaluate(val_dataset_for_evaluation)

# The 'evaluate' method returns the loss followed by the metrics in the order you compiled them.
loss = evaluation_results[0]
binary_accuracy = evaluation_results[1]
auc = evaluation_results[2]

print(f"Test Loss: {loss:.4f}")
print(f"Test Binary Accuracy: {binary_accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")

Model loaded successfully!


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 6, 13, 13, 7)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 6, 128)              │          94,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cls_out (Dense)                      │ (None, 507)                 │          32,955 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 3, 13, 13)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 176,765 (690.49 KB)

 Trainable params: 176,763 (690.48 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)


Evaluating model performance on the validation set...
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 43s 19ms/step - auc: 0.6707 - binary_accuracy: 0.9950 - loss: 0.0300
Test Loss: 0.0309
Test Binary Accuracy: 0.9948
Test AUC: 0.6675


C:\Users\Ankit\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
